# カメラの映像を取得してみよう

ここでは、AKARIのヘッドのOAK-D Liteのカメラ映像の取得方法を説明します。

## 1. ライブラリのインストール
今回はipywidetsを使うので、まずは下記のセルを実行してipywidgetsをインストールします。

In [ ]:
!pip install ipywidgets

## 2. RGB画像の取得
まずはOAK-D LiteのRGB画像の取得を試してみましょう。  


### 2.1 OAK-Dのセットアップ
OAK-Dは使い方がやや特殊です。まず、下記のセルを実行してRGB画像を取得するためのOAK-Dのセットアップをします。


In [ ]:
import cv2
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
import threading
# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_rgb = pipeline.createColorCamera()

# preview size640x480に指定
cam_rgb.setPreviewSize(640, 480) 
cam_rgb.setInterleaved(False)

# ストリーミング名設定
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

### 2.2 OAK-D画像の取得
下記のセルを実行することで、画像を取得して表示することができます。

In [ ]:
# ディスプレイを設定
display_handle=display(None, display_id=True)
# デバイスをパイプラインに接続
with dai.Device(pipeline) as device:
    # フレームを取得して表示
    video = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    frame = video.get().getCvFrame()
    _, jpg = cv2.imencode('.jpeg', frame)
    img = Image(data=jpg.tobytes())
    display_handle.update(img)

### 2.3 RGB映像のストリーミング
映像をストリームするには、2.1実行後下記のセルを実行します。  
STOPボタンで停止します。  
**AKARIのlocalhost環境でJupyter Labを実行している場合描画によりCPU負荷が高くなり、描画ウィンドウが明滅する場合があります。**   
**その場合、別PCからweb consoleにアクセスする方法を試してみてください。**

In [ ]:
# カメラ映像をストリーミングする関数
def showVideo(button):
    with dai.Device(pipeline) as device:
        queueRgb = device.getOutputQueue("rgb", maxSize=4, blocking=False)
        frame = None
        while True:
            inRgb = queueRgb.tryGet()
            if inRgb is not None:
                frame = inRgb.getCvFrame()
            if frame is not None:
                rows, columns, _ = frame.shape
                #画像サイズを1/2にリサイズする。
                resizedFrame = cv2.resize(frame, (int(columns/2), int(rows/2)))
                _, jpg = cv2.imencode('.jpeg', resizedFrame)
                display_handle.update(Image(data=jpg.tobytes()))
            if stopButton.value==True:
                break

# ボタンウィジェットを設定する
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Stop the Video',
    icon='square'
)

# ディスプレイのウィンドウを設定する
display_handle=display(None, display_id=True)
display(stopButton)
# 映像ストリーミングを開始
thread = threading.Thread(target=showVideo, args=(stopButton,))
thread.start()

## 3. ステレオdepth画像の取得
次にOAK-D LiteのステレオRGBを用いたdepth画像の取得を試してみましょう。  

### 3.1 OAK-Dのセットアップ
まず、下記のセルを実行してステレオ画像を取得するためのOAK-Dのセットアップをします。

In [ ]:
import cv2
import numpy
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
import threading

MAX_DEPTH = 1500.0  # unit: [mm]

# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_left = pipeline.create(dai.node.MonoCamera)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
cam_right = pipeline.create(dai.node.MonoCamera)
cam_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
stereo = pipeline.create(dai.node.StereoDepth)
xout_depth = pipeline.create(dai.node.XLinkOut)

# ストリーミング名設定
xout_depth.setStreamName("depth")

# ソースとアウトプットを接続
cam_left.out.link(stereo.left)
cam_right.out.link(stereo.right)
stereo.depth.link(xout_depth.input)


### 3.2 Depth画像の取得
下記のセルを実行することで、Depth画像を取得して表示することができます。

In [ ]:
# ディスプレイを設定
display_handle=display(None, display_id=True)
# デバイスをパイプラインに接続
with dai.Device(pipeline) as device:
    video = device.getOutputQueue(name="depth", maxSize=4, blocking=False)  # type: ignore
    depth = video.get().getCvFrame()
    # フレームをカラーマップに割り当て
    depth[numpy.where(depth > MAX_DEPTH)] = 0
    norm_depth = (depth * (255 / MAX_DEPTH)).astype(numpy.uint8)
    colored_depth: numpy.ndarray = cv2.applyColorMap(norm_depth, cv2.COLORMAP_JET)
    _, jpg = cv2.imencode('.jpeg', colored_depth)
    img = Image(data=jpg.tobytes())
    display_handle.update(img)

### 3.2 Depth映像のストリーミング
映像をストリームするには、3.1実行後下記のセルを実行します。  
STOPボタンで停止します。  
**AKARIのlocalhost環境でJupyter Labを実行している場合描画によりCPU負荷が高くなり、描画ウィンドウが明滅する場合があります。**   
**その場合、別PCからweb consoleにアクセスする方法を試してみてください。**

In [ ]:
# カメラ映像をストリーミングする関数
def showVideo(button):
    with dai.Device(pipeline) as device:
        queueDepth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
        while True:
            inDepth = queueDepth.tryGet()
            depth = None
            if inDepth is not None:
                depth = inDepth.getCvFrame()
            if depth is not None:
                # フレームをカラーマップに割り当て
                depth[numpy.where(depth > MAX_DEPTH)] = 0
                norm_depth = (depth * (255 / MAX_DEPTH)).astype(numpy.uint8)
                colored_depth: numpy.ndarray = cv2.applyColorMap(norm_depth, cv2.COLORMAP_JET)
                rows, columns, _ = colored_depth.shape
                colored_depth = cv2.resize(colored_depth, (int(columns/2), int(rows/2)))
                _, jpg = cv2.imencode('.jpeg', colored_depth)
                display_handle.update(Image(data=jpg.tobytes()))
            if stopButton.value==True:
                break

# ボタンウィジェットを設定する
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Stop the Video',
    icon='square'
)

# ディスプレイのウィンドウを設定する
display_handle=display(None, display_id=True)
display(stopButton)
# 映像ストリーミングを開始
thread = threading.Thread(target=showVideo, args=(stopButton,))
thread.start()

## 最後に
以上でカメラ画像取得のチュートリアルは終わりです。  
OAK-Dの使い方はここでは説明していない内容が多数あります。詳しくは[depthaiのマニュアル](https://docs.luxonis.com/en/latest/)をご覧ください。  
AKARIのチュートリアルは以上となります。